<a href="https://colab.research.google.com/github/minhthy1016/Credit-Innovators/blob/minhthy1016-patch-1/mailchimp_content.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mailchimp3

     |████████████████████████████████| 102kB 4.3MB/s 


In [ ]:
!pip install bs4

Just print result of **all_content**

In [ ]:
import datetime

import csv
import json
# import subprocess
# import sys


# def install(package):
# subprocess.check_call([sys.executable, "-m", "pip3", "install", package])


# install('mailchimp3==3.0.14')
import time

from mailchimp3 import MailChimp
from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup
import pandas as pd
import pandas_gbq
import itertools
from google.oauth2 import service_account
from google.auth import compute_engine
from google.cloud import storage
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/content/hhg-cdm-bcdeee60c0e1.json'
# today_date = datetime.datetime.now().strftime("%Y%m%d")



country_list = [
    {
        "country_code": "VN",
        "web_page": "Hello Bacsi"
    },
    {
        "country_code": "VN",
        "web_page": "Marry Baby-VN"
    },
    {
        "country_code": "MY",
        "web_page": "Hello Doktor"
    },
    {
        "country_code": "TH",
        "web_page": "Hello Khunmor"
    },
    {
        "country_code": "TW",
        "web_page": "Hello Yishi"
    },
    {
        "country_code": "PH",
        "web_page": "Hello Doctor"
    },
    {
        "country_code": "IN",
        "web_page": "Hello Swasthya"
    },
    {
        "country_code": "MM",
        "web_page": "Hello Sayarwon"
    },
    {
        "country_code": "KH",
        "web_page": "Hello Krupet"
    },
    {
        "country_code": "ID",
        "web_page": "Hello Sehat"
    }
]

country_code = {country['web_page']: country['country_code'] for country in country_list}
client = MailChimp(mc_api='1744313b5cc5b201df918cb4b9af764d-us11', mc_user='data@hellohealthgroup.com')


def html_parse_href(html_doc):
    try:
        soup = BeautifulSoup(html_doc, 'html.parser')
        return ', '.join([a.get('href') for a in soup.find_all('a', href=lambda x: 'http' in x and 'youtube' not in x)])
    except:
        return None


def html_parse_img(html_doc):
    try:
        soup = BeautifulSoup(html_doc, 'html.parser')
        return ', '.join([img.get('src') for img in soup.find_all('img')])
    except:
        return None


def html_parse_ytb(html_doc):
    try:
        soup = BeautifulSoup(html_doc, 'html.parser')
        return ', '.join([img.get('href') for img in soup.find_all('a', href=lambda x: "youtube" in x)])
    except:
        return None

def get_campaign():
    for i in range(10):
        try:
            campaigns = client.campaigns.all(get_all=True,
                                             fields="campaigns.id,campaigns.settings.subject_line,campaigns.settings.title,campaigns.settings.preview_text,campaigns.archive_url,campaigns.long_archive_url,campaigns.content_type,campaigns.create_time,campaigns.recipients.list_id,campaigns.recipients.list_name")
        except:
            continue
        break
    for campaign in campaigns['campaigns']:
        yield {
            'campaign_id': campaign.get('id'),
            'create_date': datetime.strptime(campaign.get('create_time'), '%Y-%m-%dT%H:%M:%S+00:00').date(),
            'create_time': campaign.get('create_time'),
            'subject_line': campaign.get('settings').get('subject_line'),
            'title': campaign.get('settings').get('title'),
            'list_id': campaign.get('recipients').get('list_id'),
            'list_name': campaign.get('recipients').get('list_name'),
            'content_type': campaign.get('content_type'),
            'archive_url': campaign.get('archive_url'),
            'long_archive_url': campaign.get('long_archive_url'),
            'country_code': country_code.get(campaign.get('recipients').get('list_name'))
        }


def get_content(campaigns):
    for campaign in campaigns[:10]:
        for i in range(10):
            try:
                content = client.campaigns.content.get(campaign_id=campaign['campaign_id'])
            #          print(content.get('plain_text'))
            except:
                continue
            break
        yield {
            'campaign_id': campaign['campaign_id'],
            'create_date': campaign['create_date'],
            'plain_text': content.get('plain_text'),
            'url': html_parse_href(content.get('html')),
            'img': html_parse_img(content.get('html')),
            'youtube': html_parse_ytb(content.get('html'))
        }

def rotate(l, n):
    return l[-n:] + l[:-n]

def main():
    credentials = service_account.Credentials.from_service_account_file(
        '/content/hhg-cdm-bcdeee60c0e1.json')
    #    credentials = compute_engine.Credentials()
    
    all_campaigns = list(get_campaign())
    all_campaigns_di = pd.DataFrame(all_campaigns).replace(r'\\n', ' ', regex=True).replace("\\r", ' ', regex=True)
    
    all_content = list(get_content(all_campaigns))
    all_content_di = pd.DataFrame(all_content).replace(r'\\n', ' ', regex=True).replace("\\r", ' ', regex=True)

    print(all_content_di)
    
 #save to csv
   # all_content_di.to_csv('results.csv', index=False, encoding='utf8')
    
main()





  campaign_id  ...                                            youtube
0  00bf120e87  ...  https://www.youtube.com/channel/UCRCgDjnjGWm4o...
1  0121c53d51  ...                                                   
2  015970e91e  ...  https://www.youtube.com/channel/UCBGg78eI4366C...
3  01af49aaee  ...                                                   
4  0236d2b1f2  ...                                                   
5  029e9f376b  ...  https://www.youtube.com/watch?v=vOgzv_UUnMo, h...
6  030a8d0d5f  ...  https://www.youtube.com/channel/UCBGg78eI4366C...
7  04bdaa5b41  ...                                                   
8  053d7558cc  ...                                                   
9  0633aae253  ...                                                   

[10 rows x 6 columns]


# global all_content_di

In [ ]:
import datetime

import csv
import json
# import subprocess
# import sys


# def install(package):
# subprocess.check_call([sys.executable, "-m", "pip3", "install", package])


# install('mailchimp3==3.0.14')
import time

from mailchimp3 import MailChimp
from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup
import pandas as pd
import pandas_gbq
import itertools
from google.oauth2 import service_account
from google.auth import compute_engine
from google.cloud import storage
import os
from google.cloud import bigquery
from google.api_core.exceptions import BadRequest

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/content/hhg-cdm-bcdeee60c0e1.json'
# today_date = datetime.datetime.now().strftime("%Y%m%d")



country_list = [
    {
        "country_code": "VN",
        "web_page": "Hello Bacsi"
    },
    {
        "country_code": "VN",
        "web_page": "Marry Baby-VN"
    },
    {
        "country_code": "MY",
        "web_page": "Hello Doktor"
    },
    {
        "country_code": "TH",
        "web_page": "Hello Khunmor"
    },
    {
        "country_code": "TW",
        "web_page": "Hello Yishi"
    },
    {
        "country_code": "PH",
        "web_page": "Hello Doctor"
    },
    {
        "country_code": "IN",
        "web_page": "Hello Swasthya"
    },
    {
        "country_code": "MM",
        "web_page": "Hello Sayarwon"
    },
    {
        "country_code": "KH",
        "web_page": "Hello Krupet"
    },
    {
        "country_code": "ID",
        "web_page": "Hello Sehat"
    }
]

country_code = {country['web_page']: country['country_code'] for country in country_list}
client = MailChimp(mc_api='1744313b5cc5b201df918cb4b9af764d-us11', mc_user='data@hellohealthgroup.com')


def html_parse_href(html_doc):
    try:
        soup = BeautifulSoup(html_doc, 'html.parser')
        return ', '.join([a.get('href') for a in soup.find_all('a', href=lambda x: 'http' in x and 'youtube' not in x)])
    except:
        return None


def html_parse_img(html_doc):
    try:
        soup = BeautifulSoup(html_doc, 'html.parser')
        return ', '.join([img.get('src') for img in soup.find_all('img')])
    except:
        return None


def html_parse_ytb(html_doc):
    try:
        soup = BeautifulSoup(html_doc, 'html.parser')
        return ', '.join([img.get('href') for img in soup.find_all('a', href=lambda x: "youtube" in x)])
    except:
        return None

def get_campaign():
    for i in range(10):
        try:
            campaigns = client.campaigns.all(get_all=True,
                                             fields="campaigns.id,campaigns.settings.subject_line,campaigns.settings.title,campaigns.settings.preview_text,campaigns.archive_url,campaigns.long_archive_url,campaigns.content_type,campaigns.create_time,campaigns.recipients.list_id,campaigns.recipients.list_name")
        except:
            continue
        break
    for campaign in campaigns['campaigns']:
        yield {
            'campaign_id': campaign.get('id'),
            'create_date': datetime.strptime(campaign.get('create_time'), '%Y-%m-%dT%H:%M:%S+00:00').date(),
            'create_time': campaign.get('create_time'),
            'subject_line': campaign.get('settings').get('subject_line'),
            'title': campaign.get('settings').get('title'),
            'list_id': campaign.get('recipients').get('list_id'),
            'list_name': campaign.get('recipients').get('list_name'),
            'content_type': campaign.get('content_type'),
            'archive_url': campaign.get('archive_url'),
            'long_archive_url': campaign.get('long_archive_url'),
            'country_code': country_code.get(campaign.get('recipients').get('list_name'))
        }


def get_content(campaigns):
    for campaign in campaigns[:10]:
        for i in range(10):
            try:
                content = client.campaigns.content.get(campaign_id=campaign['campaign_id'])
            #          print(content.get('plain_text'))
            except:
                continue
            break
        yield {
            'campaign_id': campaign['campaign_id'],
            'create_date': campaign['create_date'],
            'plain_text': content.get('plain_text'),
            'url': html_parse_href(content.get('html')),
            'img': html_parse_img(content.get('html')),
            'youtube': html_parse_ytb(content.get('html'))
        }

def rotate(l, n):
    return l[-n:] + l[:-n]

def main():
    credentials = service_account.Credentials.from_service_account_file(
        '/content/hhg-cdm-bcdeee60c0e1.json')
    #    credentials = compute_engine.Credentials()
    job_config = bigquery.LoadJobConfig()
    job_config.allow_quoted_newlines = True
    all_campaigns = list(get_campaign())
    all_campaigns_di = pd.DataFrame(all_campaigns).replace(r'\\n', ' ', regex=True).replace("\\r", ' ', regex=True)

    all_content = list(get_content(all_campaigns))
    global all_content_di
    all_content_di = pd.DataFrame(all_content).replace(r'\\n', ' ', regex=True).replace("\\r", ' ', regex=True)

    print(all_content_di)
    
 #save to csv
   # all_content_di.to_csv('results.csv', index=False, encoding='utf8')
    
main()




  campaign_id  ...                                            youtube
0  00bf120e87  ...  https://www.youtube.com/channel/UCRCgDjnjGWm4o...
1  0121c53d51  ...                                                   
2  015970e91e  ...  https://www.youtube.com/channel/UCBGg78eI4366C...
3  01af49aaee  ...                                                   
4  0236d2b1f2  ...                                                   
5  029e9f376b  ...  https://www.youtube.com/watch?v=vOgzv_UUnMo, h...
6  030a8d0d5f  ...  https://www.youtube.com/channel/UCBGg78eI4366C...
7  04bdaa5b41  ...                                                   
8  053d7558cc  ...                                                   
9  0633aae253  ...                                                   

[10 rows x 6 columns]


In [ ]:
#Python code example to load data in bigquery
client = bigquery.Client('project-id-b4f8d566')
dataset_id = 'census-ds'

dataset_ref = client.dataset(dataset_id)
job_config = bigquery.LoadJobConfig()
job_config.allow_quoted_newlines = True
job_config.schema = [
    bigquery.SchemaField("id", "INTEGER","REQUIRED"),
    bigquery.SchemaField("code", "STRING" ,"NULLABLE"),
    bigquery.SchemaField("answer", "STRING","NULLABLE")
]
job_config.skip_leading_rows = 1
# The source format defaults to CSV, so the line below is optional.
job_config.source_format = bigquery.SourceFormat.CSV
uri = "gs://mybucket/text.csv"

load_job = client.load_table_from_uri(
    uri, dataset_ref.table("census_text"), job_config=job_config
)  # API request
print("Starting job {}".format(load_job.job_id))

try:
    load_job.result()  # Waits for table load to complete.
    print("Job finished.")

    destination_table = client.get_table(dataset_ref.table("census_text"))
    print("Loaded {} rows.".format(destination_table.num_rows))
except BadRequest as e:
    for e in load_job.errors:
        print('ERROR: {}'.format(e['message']))



NotFound: ignored

In [ ]:
pip install \
 apache-airflow==1.10.10 \
 --constraint \
        https://raw.githubusercontent.com/apache/airflow/1.10.10/requirements/requirements-python3.7.txt

In [ ]:
import datetime
from airflow import DAG
from airflow.utils.dates import days_ago
from airflow.operators.dummy_operator import DummyOperator

from airflow.operators.python_operator import PythonOperator
from airflow.contrib.operators import gcs_to_bq
import csv
import json
# import subprocess
# import sys


# def install(package):
# subprocess.check_call([sys.executable, "-m", "pip3", "install", package])


# install('mailchimp3==3.0.14')
import time

from mailchimp3 import MailChimp
from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup
import pandas as pd
import pandas_gbq
import itertools
from google.oauth2 import service_account
from google.auth import compute_engine
from google.cloud import storage
import os
from google.cloud import bigquery
from google.api_core.exceptions import BadRequest

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/content/hhg-cdm-bcdeee60c0e1.json'
default_dag_args = {
    'owner': 'data@hellohealthgroup.com',
    'depends_on_past': False,
    'start_date': days_ago(1),
    'retries': 2,
    # 'retry_delay': datetime.timedelta(minutes=5),
    'project_id': 'hhg-cdm'

}
with DAG(dag_id='mc_test', description='mc_test',
         default_args=default_dag_args) as dag:
    ####Dummy Variable For Start
    START = DummyOperator(task_id='START')
    ####Dummy Variable For End
    FINISH = DummyOperator(task_id='FINISH', depends_on_past=True)
    # logging.error('table name: '+table_name)

country_list = [
    {
        "country_code": "VN",
        "web_page": "Hello Bacsi"
    },
    {
        "country_code": "VN",
        "web_page": "Marry Baby-VN"
    },
    {
        "country_code": "MY",
        "web_page": "Hello Doktor"
    },
    {
        "country_code": "TH",
        "web_page": "Hello Khunmor"
    },
    {
        "country_code": "TW",
        "web_page": "Hello Yishi"
    },
    {
        "country_code": "PH",
        "web_page": "Hello Doctor"
    },
    {
        "country_code": "IN",
        "web_page": "Hello Swasthya"
    },
    {
        "country_code": "MM",
        "web_page": "Hello Sayarwon"
    },
    {
        "country_code": "KH",
        "web_page": "Hello Krupet"
    },
    {
        "country_code": "ID",
        "web_page": "Hello Sehat"
    }
]

country_code = {country['web_page']: country['country_code'] for country in country_list}
client = MailChimp(mc_api='1744313b5cc5b201df918cb4b9af764d-us11', mc_user='data@hellohealthgroup.com')


def html_parse_href(html_doc):
    try:
        soup = BeautifulSoup(html_doc, 'html.parser')
        return ', '.join([a.get('href') for a in soup.find_all('a', href=lambda x: 'http' in x and 'youtube' not in x)])
    except:
        return None


def html_parse_img(html_doc):
    try:
        soup = BeautifulSoup(html_doc, 'html.parser')
        return ', '.join([img.get('src') for img in soup.find_all('img')])
    except:
        return None


def html_parse_ytb(html_doc):
    try:
        soup = BeautifulSoup(html_doc, 'html.parser')
        return ', '.join([img.get('href') for img in soup.find_all('a', href=lambda x: "youtube" in x)])
    except:
        return None


def get_campaign():
    for i in range(10):
        try:
            campaigns = client.campaigns.all(get_all=True,
                                             fields="campaigns.id,campaigns.settings.subject_line,campaigns.settings.title,campaigns.settings.preview_text,campaigns.archive_url,campaigns.long_archive_url,campaigns.content_type,campaigns.create_time,campaigns.recipients.list_id,campaigns.recipients.list_name")
        except:
            continue
        break
    for campaign in campaigns['campaigns']:
        yield {
            'campaign_id': campaign.get('id'),
            'create_date': str(datetime.strptime(campaign.get('create_time'), '%Y-%m-%dT%H:%M:%S+00:00').date()),
            'create_time': str(campaign.get('create_time')),
            'subject_line': campaign.get('settings').get('subject_line'),
            'title': campaign.get('settings').get('title'),
            'list_id': campaign.get('recipients').get('list_id'),
            'list_name': campaign.get('recipients').get('list_name'),
            'content_type': campaign.get('content_type'),
            'archive_url': campaign.get('archive_url'),
            'long_archive_url': campaign.get('long_archive_url'),
            'country_code': country_code.get(campaign.get('recipients').get('list_name'))
        }


def get_content(campaigns):
    for campaign in campaigns[:10]:
        for i in range(10):
            try:
                content = client.campaigns.content.get(campaign_id=campaign['campaign_id'])
            #          print(content.get('plain_text'))
            except:
                continue
            break
        yield {
            'campaign_id': campaign['campaign_id'],
            'create_date': str(campaign['create_date']),
            'plain_text': content.get('plain_text'),
            'url': html_parse_href(content.get('html')),
            'img': html_parse_img(content.get('html')),
            'youtube': html_parse_ytb(content.get('html'))
        }


def get_subscriber(lists):
    #    print(lists.get('id'), lists.get('stats').get('member_count'), lists.get('stats').get('cleaned_count'),lists.get('stats').get('unsubscribe_count'),lists.get('stats').get('merge_field_count') )
    for i in range((lists.get('stats').get('member_count') + lists.get('stats').get('cleaned_count') + lists.get(
            'stats').get('unsubscribe_count') + lists.get('stats').get('merge_field_count')) // 1000 + 1):
        for k in range(10):
            try:
                subscriber = client.lists.members.all(list_id=lists.get('id'), count=1000, offset=i * 1000,
                                                      fields="members.list_id,members.id,members.unique_email_id,members.timestamp_opt,members.email_address,members.status,members.tags,members.timestamp_signup,members.last_changed,members.location.country_code").get(
                    'members')
            except:
                continue
            break
        yield subscriber


def rotate(l, n):
    return l[-n:] + l[:-n]



def streaming_to_bq(table_id, rows_to_insert, schema):
    # delete table
    client = bigquery.Client.from_service_account_json(r'/content/hhg-cdm-bcdeee60c0e1.json')
    client.delete_table(table_id, not_found_ok=True)  # Make an API request.
    print("Deleted table '{}'.".format(table_id))

    # create table
    table = bigquery.Table(table_id, schema=schema)
    table = client.create_table(table)  # Make an API request.
    print("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))

    # streaming data
    errors = client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
    if errors == []:
        print("New rows have been added for: ", table_id)
    else:
        print("Encountered errors while inserting rows: {}".format(errors))


#def main():
credentials = service_account.Credentials.from_service_account_file(
        r'/content/hhg-cdm-bcdeee60c0e1.json')
all_campaigns = list(get_campaign())
all_content = list(get_content(all_campaigns))
content_schema = [
        bigquery.SchemaField("campaign_id", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("create_date", "DATE", mode="NULLABLE"),
        bigquery.SchemaField("plain_text", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("url", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("img", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("youtube", "STRING", mode="NULLABLE"),
]
    #streaming_to_bq('sandbox-247611.backup_test.content', all_content, content_schema)

campaigns_schema = [
        bigquery.SchemaField("campaign_id", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("create_date", "DATE", mode="NULLABLE"),
        bigquery.SchemaField("create_time", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("subject_line", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("title", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("list_id", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("list_name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("content_type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("archive_url", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("long_archive_url", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("country_code", "STRING", mode="NULLABLE"),
]
    #streaming_to_bq('sandbox-247611.backup_test.mailchimp_campaign', all_campaigns, campaigns_schema)


run_hhg_mailchimp_all_content_di = PythonOperator(
    task_id='hhg_mailchimp_all_content_di',
    provide_context=False,
    depends_on_past=False,
    python_callable=streaming_to_bq,
  #  op_kwargs={'table_id': 'hhg-ods.test.content', 'rows_to_insert': all_content, 'schema': content_schema},
    op_kwargs={'table_id': 'sandbox-247611.backup_test.content', 'rows_to_insert': all_content, 'schema': content_schema},
    dag=dag
)

run_hhg_mailchimp_all_campaign_di = PythonOperator(
    task_id='hhg_mailchimp_all_campaign_di',
    provide_context=False,
    depends_on_past=False,
    python_callable=streaming_to_bq,
    op_kwargs={'table_id': 'sandbox-247611.backup_test.mailchimp_campaign', 'rows_to_insert': all_campaigns, 'schema': campaigns_schema},
    dag=dag
)


# Define DAG :
START >>[run_hhg_mailchimp_all_content_di,run_hhg_mailchimp_all_campaign_di] >> FINISH


[2020-11-19 02:57:40,707] {mailchimpclient.py:97} INFO - GET Request: https://us11.api.mailchimp.com/3.0/campaigns?offset=0&count=500&fields=campaigns.id%2Ccampaigns.settings.subject_line%2Ccampaigns.settings.title%2Ccampaigns.settings.preview_text%2Ccampaigns.archive_url%2Ccampaigns.long_archive_url%2Ccampaigns.content_type%2Ccampaigns.create_time%2Ccampaigns.recipients.list_id%2Ccampaigns.recipients.list_name%2Ctotal_items
[2020-11-19 02:57:43,576] {mailchimpclient.py:104} INFO - GET Response: 200
[2020-11-19 02:57:43,603] {mailchimpclient.py:97} INFO - GET Request: https://us11.api.mailchimp.com/3.0/campaigns?offset=500&count=500&fields=campaigns.id%2Ccampaigns.settings.subject_line%2Ccampaigns.settings.title%2Ccampaigns.settings.preview_text%2Ccampaigns.archive_url%2Ccampaigns.long_archive_url%2Ccampaigns.content_type%2Ccampaigns.create_time%2Ccampaigns.recipients.list_id%2Ccampaigns.recipients.list_name%2Ctotal_items
[2020-11-19 02:57:45,898] {mailchimpclient.py:104} INFO - GET Re

<Task(DummyOperator): FINISH>

## Add tags_di + ETL + stream result to BQ


stream data result : all_campaigns, all_content là ghi đè lên bảng cũ và không có dữ liệu (Why???) 
Nên đành thử chuyển về DataFrame và ETL, sau đó stream kq về GCS/BQ?? 

In [ ]:
from pandas.io.common import is_url

In [ ]:
import datetime
from airflow import DAG
from airflow.utils.dates import days_ago
from airflow.operators.dummy_operator import DummyOperator

from airflow.operators.python_operator import PythonOperator
from airflow.contrib.operators import gcs_to_bq
import csv
import json
# import subprocess
# import sys


# def install(package):
# subprocess.check_call([sys.executable, "-m", "pip3", "install", package])


# install('mailchimp3==3.0.14')
import time

from mailchimp3 import MailChimp
from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader as web
import numpy as np
import pandas_gbq
import itertools
from google.oauth2 import service_account
from google.auth import compute_engine
from google.cloud import storage
import os
from google.cloud import bigquery
from google.api_core.exceptions import BadRequest

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/content/hhg-cdm-bcdeee60c0e1.json'
default_dag_args = {
    'owner': 'data@hellohealthgroup.com',
    'depends_on_past': False,
    'start_date': days_ago(1),
    'retries': 2,
    # 'retry_delay': datetime.timedelta(minutes=5),
    'project_id': 'hhg-cdm'

}
with DAG(dag_id='mc_test', description='mc_test',
         default_args=default_dag_args) as dag:
    ####Dummy Variable For Start
    START = DummyOperator(task_id='START')
    ####Dummy Variable For End
    FINISH = DummyOperator(task_id='FINISH', depends_on_past=True)
    # logging.error('table name: '+table_name)

country_list = [
    {
        "country_code": "VN",
        "web_page": "Hello Bacsi"
    },
    {
        "country_code": "VN",
        "web_page": "Marry Baby-VN"
    },
    {
        "country_code": "MY",
        "web_page": "Hello Doktor"
    },
    {
        "country_code": "TH",
        "web_page": "Hello Khunmor"
    },
    {
        "country_code": "TW",
        "web_page": "Hello Yishi"
    },
    {
        "country_code": "PH",
        "web_page": "Hello Doctor"
    },
    {
        "country_code": "IN",
        "web_page": "Hello Swasthya"
    },
    {
        "country_code": "MM",
        "web_page": "Hello Sayarwon"
    },
    {
        "country_code": "KH",
        "web_page": "Hello Krupet"
    },
    {
        "country_code": "ID",
        "web_page": "Hello Sehat"
    }
]

country_code = {country['web_page']: country['country_code'] for country in country_list}
client = MailChimp(mc_api='1744313b5cc5b201df918cb4b9af764d-us11', mc_user='data@hellohealthgroup.com')


def html_parse_href(html_doc):
    try:
        soup = BeautifulSoup(html_doc, 'html.parser')
        return ', '.join([a.get('href') for a in soup.find_all('a', href=lambda x: 'http' in x and 'youtube' not in x)])
    except:
        return None


def html_parse_img(html_doc):
    try:
        soup = BeautifulSoup(html_doc, 'html.parser')
        return ', '.join([img.get('src') for img in soup.find_all('img')])
    except:
        return None


def html_parse_ytb(html_doc):
    try:
        soup = BeautifulSoup(html_doc, 'html.parser')
        return ', '.join([img.get('href') for img in soup.find_all('a', href=lambda x: "youtube" in x)])
    except:
        return None


def get_campaign():
    for i in range(10):
        try:
            campaigns = client.campaigns.all(get_all=True,
                                             fields="campaigns.id,campaigns.settings.subject_line,campaigns.settings.title,campaigns.settings.preview_text,campaigns.archive_url,campaigns.long_archive_url,campaigns.content_type,campaigns.create_time,campaigns.recipients.list_id,campaigns.recipients.list_name")
        except:
            continue
        break
    for campaign in campaigns['campaigns']:
        yield {
            'campaign_id': campaign.get('id'),
            'create_date': str(datetime.strptime(campaign.get('create_time'), '%Y-%m-%dT%H:%M:%S+00:00').date()),
            'create_time': str(campaign.get('create_time')),
            'subject_line': campaign.get('settings').get('subject_line'),
            'title': campaign.get('settings').get('title'),
            'list_id': campaign.get('recipients').get('list_id'),
            'list_name': campaign.get('recipients').get('list_name'),
            'content_type': campaign.get('content_type'),
            'archive_url': campaign.get('archive_url'),
            'long_archive_url': campaign.get('long_archive_url'),
            'country_code': country_code.get(campaign.get('recipients').get('list_name'))
        }


def get_content(campaigns):
    for campaign in campaigns[:10]:
        for i in range(10):
            try:
                content = client.campaigns.content.get(campaign_id=campaign['campaign_id'])
            #          print(content.get('plain_text'))
            except:
                continue
            break
        yield {
            'campaign_id': campaign['campaign_id'],
            'create_date': str(campaign['create_date']),
            'plain_text': content.get('plain_text'),
            'url': html_parse_href(content.get('html')),
            'img': html_parse_img(content.get('html')),
            'youtube': html_parse_ytb(content.get('html'))
        }


def get_subscriber(lists):
    #    print(lists.get('id'), lists.get('stats').get('member_count'), lists.get('stats').get('cleaned_count'),lists.get('stats').get('unsubscribe_count'),lists.get('stats').get('merge_field_count') )
    for i in range((lists.get('stats').get('member_count') + lists.get('stats').get('cleaned_count') + lists.get(
            'stats').get('unsubscribe_count') + lists.get('stats').get('merge_field_count')) // 1000 + 1):
        for k in range(10):
            try:
                subscriber = client.lists.members.all(list_id=lists.get('id'), count=1000, offset=i * 1000,
                                                      fields="members.list_id,members.id,members.unique_email_id,members.timestamp_opt,members.email_address,members.status,members.tags,members.timestamp_signup,members.last_changed,members.location.country_code").get(
                    'members')
            except:
                continue
            break
        yield subscriber


def rotate(l, n):
    return l[-n:] + l[:-n]

def get_subscriber(lists):
    #    print(lists.get('id'), lists.get('stats').get('member_count'), lists.get('stats').get('cleaned_count'),lists.get('stats').get('unsubscribe_count'),lists.get('stats').get('merge_field_count') )
    for i in range((lists.get('stats').get('member_count') + lists.get('stats').get('cleaned_count') + lists.get(
            'stats').get('unsubscribe_count') + lists.get('stats').get('merge_field_count')) // 1000 + 1):
        for k in range(10):
            try:
                subscriber = client.lists.members.all(list_id=lists.get('id'), count=1000, offset=i * 1000,
                                                      fields="members.list_id,members.id,members.unique_email_id,members.timestamp_opt,members.email_address,members.status,members.tags,members.timestamp_signup,members.last_changed,members.location.country_code").get(
                    'members')
            except:
                continue
            break
        yield subscriber

def get_all_tags(all_list):
    for lists in all_list['lists']:
        subscriber_list = list(get_subscriber(lists))
        subscriber = list(itertools.chain.from_iterable(subscriber_list))
        yield subscriber



def streaming_to_bq(table_id, rows_to_insert, schema):
    # delete table
    client = bigquery.Client.from_service_account_json(r'/content/hhg-cdm-bcdeee60c0e1.json')
    client.delete_table(table_id, not_found_ok=True)  # Make an API request.
    print("Deleted table '{}'.".format(table_id))

    # create table
    table = bigquery.Table(table_id, schema=schema)
    table = client.create_table(table)  # Make an API request.
    print("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))

    # streaming data
    errors = client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
    if errors == []:
        print("New rows have been added for: ", table_id)
    else:
        print("Encountered errors while inserting rows: {}".format(errors))


#def main():
credentials = service_account.Credentials.from_service_account_file(
        r'/content/hhg-cdm-bcdeee60c0e1.json')
all_campaigns = list(get_campaign())
global all_content
all_content = list(get_content(all_campaigns))
all_list = client.lists.all(get_all=True, fields='lists.id,lists.stats')
tags = list(itertools.chain.from_iterable(get_all_tags(all_list)))

content_schema = [
        bigquery.SchemaField("campaign_id", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("create_date", "DATE", mode="NULLABLE"),
        bigquery.SchemaField("plain_text", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("url", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("img", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("youtube", "STRING", mode="NULLABLE"),
]
streaming_to_bq('sandbox-247611.backup_test.content', all_content, content_schema)    
#it write truncate and no data

campaigns_schema = [
        bigquery.SchemaField("campaign_id", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("create_date", "DATE", mode="NULLABLE"),
        bigquery.SchemaField("create_time", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("subject_line", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("title", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("list_id", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("list_name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("content_type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("archive_url", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("long_archive_url", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("country_code", "STRING", mode="NULLABLE"),
]
streaming_to_bq('sandbox-247611.backup_test.mailchimp_campaign', all_campaigns, campaigns_schema)

tags_schema = [
        bigquery.SchemaField("list_id", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("id", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("email_address", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("unique_email_id", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("status", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("timestamp_signup", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("timestamp_opt", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("last_changed", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("location", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("tags", "STRING", mode="NULLABLE"),

]

all_campaigns_di = pd.DataFrame(all_campaigns).replace(r'\\n', ' ', regex=True).replace("\\r", ' ', regex=True)

all_content_di = pd.DataFrame(all_content).replace(r'\\n', ' ', regex=True).replace("\\r", ' ', regex=True)

all_list = client.lists.all(get_all=True, fields='lists.id,lists.stats')
tags = list(itertools.chain.from_iterable(get_all_tags(all_list)))
tags_df = pd.DataFrame(tags)
tags_df['location'] = tags_df['location'].apply(lambda x: x.get('country_code'))
tags_df['tags'] = tags_df['tags'].apply(lambda tags: ','.join([tag.get('name') for tag in tags]))
new_col = rotate(list(tags_df.columns), 1)
tags_di = tags_df[new_col].replace(r'\\n', ' ', regex=True).replace("\\r", ' ', regex=True)
    
    
### ETL Process
global result_2
result_1 = pd.merge(all_campaigns_di,all_content_di, on=['campaign_id','create_date'])
result_2 = pd.merge(result_1,tags_di, on= ['list_id'])
result_2.head(5)


[2020-11-19 03:16:48,383] {mailchimpclient.py:97} INFO - GET Request: https://us11.api.mailchimp.com/3.0/campaigns?offset=0&count=500&fields=campaigns.id%2Ccampaigns.settings.subject_line%2Ccampaigns.settings.title%2Ccampaigns.settings.preview_text%2Ccampaigns.archive_url%2Ccampaigns.long_archive_url%2Ccampaigns.content_type%2Ccampaigns.create_time%2Ccampaigns.recipients.list_id%2Ccampaigns.recipients.list_name%2Ctotal_items
[2020-11-19 03:16:51,101] {mailchimpclient.py:104} INFO - GET Response: 200
[2020-11-19 03:16:51,114] {mailchimpclient.py:97} INFO - GET Request: https://us11.api.mailchimp.com/3.0/campaigns?offset=500&count=500&fields=campaigns.id%2Ccampaigns.settings.subject_line%2Ccampaigns.settings.title%2Ccampaigns.settings.preview_text%2Ccampaigns.archive_url%2Ccampaigns.long_archive_url%2Ccampaigns.content_type%2Ccampaigns.create_time%2Ccampaigns.recipients.list_id%2Ccampaigns.recipients.list_name%2Ctotal_items
[2020-11-19 03:16:53,493] {mailchimpclient.py:104} INFO - GET Re

ImportError: ignored

  campaign_id create_date  ... location          tags
0  00bf120e87  2020-11-13  ...           Covidchecker
1  00bf120e87  2020-11-13  ...       IN  Covidchecker
2  00bf120e87  2020-11-13  ...       IN              
3  00bf120e87  2020-11-13  ...       IN              
4  00bf120e87  2020-11-13  ...                       

[5 rows x 24 columns]

In [ ]:
result_2.groupby('create_date')['country_code'].count()
import datetime
from datetime import datetime, timedelta
ytd=datetime.today()-timedelta(1)
ytd=str(ytd.date())

##Add a new column, but not necessary anyway. 
#df1 = result_1.assign(web_page=result_1.apply(web_page, axis=1))
#df1.head(3)
## Rename columns 
global df1 
df1 = result_2.rename({'create_date': 'date', 'list_name':'web_page'}, axis=1)
df1['date'] = ytd
df1
#df2['date']= ytd

# WHERE date = yesterday 


ImportError: ignored

       campaign_id        date  ... location                                 tags
0       00bf120e87  2020-11-18  ...                                  Covidchecker
1       00bf120e87  2020-11-18  ...       IN                         Covidchecker
2       00bf120e87  2020-11-18  ...       IN                                     
3       00bf120e87  2020-11-18  ...       IN                                     
4       00bf120e87  2020-11-18  ...                                              
...            ...         ...  ...      ...                                  ...
308346  0236d2b1f2  2020-11-18  ...           HHG,Newsletter Subscriber Main Page
308347  0236d2b1f2  2020-11-18  ...           HHG,Newsletter Subscriber Main Page
308348  0236d2b1f2  2020-11-18  ...           HHG,Newsletter Subscriber Main Page
308349  0236d2b1f2  2020-11-18  ...           HHG,Newsletter Subscriber Main Page
308350  0236d2b1f2  2020-11-18  ...           HHG,Newsletter Subscriber Main Page

[308351 rows x 

In [ ]:
df1.to_csv('test_mailchimp.csv', index= False, encoding='utf-8-sig')


ImportError: ignored

In [ ]:
pip install UnicodeWriter

ERROR: Could not find a version that satisfies the requirement UnicodeWriter (from versions: none)
ERROR: No matching distribution found for UnicodeWriter


In [ ]:
from pandas import io

# Working Script to REPLACE CONTENT

In [ ]:
import datetime
from airflow import DAG
from airflow.utils.dates import days_ago
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator
from airflow.contrib.operators import gcs_to_bq
import csv
import json
from pprint import pprint
# import subprocess
# import sys
# def install(package):
# subprocess.check_call([sys.executable, "-m", "pip3", "install", package])
# install('mailchimp3==3.0.14')
import time
from mailchimp3 import MailChimp
from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup
import pandas as pd
import pandas_gbq
import itertools
from google.oauth2 import service_account
from google.auth import compute_engine
from google.cloud import storage
import os
from google.cloud import bigquery
from google.api_core.exceptions import BadRequest
from pprint import pprint
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/content/hhg-cdm-bcdeee60c0e1.json'

default_dag_args = {'ownder': 'data@hellohealthgroup,com',
                    'depend_on_past': False, 
                    'start_date': days_ago(1),
                    'retries': 2,
                    'project_id': 'hhg-cdm'}

with DAG(dag_id='mailchimp_backfill', description='mailchimp_backfill',
         default_args=default_dag_args) as dag:
    ####Dummy Variable For Start
    START = DummyOperator(task_id='START')
    ####Dummy Variable For End
    FINISH = DummyOperator(task_id='FINISH', depends_on_past=True)
    # logging.error('table name: '+table_name)

country_list = [
    {
        "country_code": "VN",
        "web_page": "Hello Bacsi"
    },
    {
        "country_code": "VN",
        "web_page": "Marry Baby-VN"
    },
    {
        "country_code": "MY",
        "web_page": "Hello Doktor"
    },
    {
        "country_code": "TH",
        "web_page": "Hello Khunmor"
    },
    {
        "country_code": "TW",
        "web_page": "Hello Yishi"
    },
    {
        "country_code": "PH",
        "web_page": "Hello Doctor"
    },
    {
        "country_code": "IN",
        "web_page": "Hello Swasthya"
    },
    {
        "country_code": "MM",
        "web_page": "Hello Sayarwon"
    },
    {
        "country_code": "KH",
        "web_page": "Hello Krupet"
    },
    {
        "country_code": "ID",
        "web_page": "Hello Sehat"
    }
]

country_code = {country['web_page']: country['country_code'] for country in country_list}
client = MailChimp(mc_api='1744313b5cc5b201df918cb4b9af764d-us11', mc_user='data@hellohealthgroup.com')

def html_parse_href(html_doc):
    try:
        soup = BeautifulSoup(html_doc, 'html.parser')
        return ', '.join([a.get('href') for a in soup.find_all('a', href=lambda x: 'http' in x and 'youtube' not in x)])
    except:
        return None

def html_parse_img(html_doc):
    try:
        soup = BeautifulSoup(html_doc, 'html.parser')
        return ', '.join([img.get('src') for img in soup.find_all('img')])
    except:
        return None

def html_parse_ytb(html_doc):
    try:
        soup = BeautifulSoup(html_doc, 'html.parser')
        return ', '.join([img.get('href') for img in soup.find_all('a', href=lambda x: "youtube" in x)])
    except:
        return None

def get_campaign():
    for i in range(10):
        try:
            campaigns = client.campaigns.all(get_all=True,
                                             fields="campaigns.id,campaigns.settings.subject_line,campaigns.settings.title,campaigns.settings.preview_text,campaigns.archive_url,campaigns.long_archive_url,campaigns.content_type,campaigns.create_time,campaigns.recipients.list_id,campaigns.recipients.list_name")
        except:
            continue
        break
    for campaign in campaigns['campaigns']:
        yield {
            'campaign_id': campaign.get('id'),
            'create_date': str(datetime.strptime(campaign.get('create_time'), '%Y-%m-%dT%H:%M:%S+00:00').date()),
            'create_time': str(campaign.get('create_time')),
            'subject_line': campaign.get('settings').get('subject_line'),
            'title': campaign.get('settings').get('title'),
            'list_id': campaign.get('recipients').get('list_id'),
            'list_name': campaign.get('recipients').get('list_name'),
            'content_type': campaign.get('content_type'),
            'archive_url': campaign.get('archive_url'),
            'long_archive_url': campaign.get('long_archive_url'),
            'country_code': country_code.get(campaign.get('recipients').get('list_name'))
        }

def get_content(campaigns):
    for campaign in campaigns[:10]:
        for i in range(10):
            try:
                content = client.campaigns.content.get(campaign_id=campaign['campaign_id'])
            #          print(content.get('plain_text'))
            except:
                continue
            break
        yield {
            'campaign_id': campaign['campaign_id'],
            'create_date': str(campaign['create_date']),
            'plain_text': content.get('plain_text'),
            'url': html_parse_href(content.get('html')),
            'img': html_parse_img(content.get('html')),
            'youtube': html_parse_ytb(content.get('html'))
        }

def rotate(l, n):
    return l[-n:] + l[:-n]

def streaming_to_bq(table_id,rows_to_insert,schema):
    #delete table
    client = bigquery.Client.from_service_account_json(r'/content/hhg-cdm-bcdeee60c0e1.json')

    client.delete_table(table_id, not_found_ok=True)  # Make an API request.
    print("Deleted table '{}'.".format(table_id))

    #create table
    table = bigquery.Table(table_id, schema=schema)
    table = client.create_table(table)  # Make an API request.
    print("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))

    #streaming data
    errors = client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
    if errors == []:
        print("New rows have been added for: ", table_id)
    else:
        print("Encountered errors while inserting rows: {}".format(errors))


def main():
    credentials = service_account.Credentials.from_service_account_file(
        r'/content/hhg-cdm-bcdeee60c0e1.json')
    all_campaigns = list(get_campaign())
    pprint(all_campaigns)
    
    all_content = list(get_content(all_campaigns))
    pprint(all_content)

    content_schema=[
    bigquery.SchemaField("campaign_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("create_date", "DATE", mode="NULLABLE"),
    bigquery.SchemaField("plain_text", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("url", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("img", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("youtube", "STRING", mode="NULLABLE"),
    ]
    streaming_to_bq('hhg-ods.test.content', all_content, content_schema)

    campaigns_schema=[
    bigquery.SchemaField("campaign_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("create_date", "DATE", mode="NULLABLE"),
    bigquery.SchemaField("create_time", "TIMESTAMP", mode="NULLABLE"),
    bigquery.SchemaField("subject_line", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("title", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("list_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("list_name", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("content_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("archive_url", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("long_archive_url", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("country_code", "STRING", mode="NULLABLE"),
    ]
    streaming_to_bq('sandbox-247611.backup_test.mailchimp_campaign', all_campaigns, campaigns_schema)

main()

'''
run_hhg_mailchimp_all_content_di = PythonOperator(
    task_id='hhg_mailchimp_all_content_di',
    provide_context=False,
    depends_on_past=False,
    python_callable=streaming_to_bq,
    op_kwargs={'table_id': 'hhg-ods.test.content', 'rows_to_insert': all_content, 'schema': content_schema}
    dag=dag,
)
​
​
​
load_csv = gcs_to_bq.GoogleCloudStorageToBigQueryOperator(
    task_id='gcs_to_bq',
    bucket='hhg_mailchimp_campaign',
    source_objects=['all_campaigns_di.csv'],
    destination_project_dataset_table='hhg_lead_mailchimp.hhg_mailchimp_all_content_di',
    job_config = bigquery.LoadJobConfig(),
    job_config.allow_quoted_newlines = True,
    job_config.schema_fields=[
        {'name': 'campaign_id', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'create_date', 'type': 'DATE', 'mode': 'NULLABLE'},
        {'name': 'plain_text', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'url', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'title', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'img', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'youtube', 'type': 'STRING', 'mode': 'NULLABLE'},
​
    ],
    job_config.skip_leading_rows = 1,
    write_disposition='WRITE_TRUNCATE',
    dag=dag)
'''

# # Define DAG :
# #START >> run_hhg_mailchimp_all_content_di >> insert_hhg_mailchimp_df >> FINISH


Streaming output truncated to the last 5000 lines.
  'country_code': 'IN',
  'create_date': '2020-08-06',
  'create_time': '2020-08-06T05:39:13+00:00',
  'list_id': '12cb199be2',
  'list_name': 'Hello Swasthya',
  'long_archive_url': 'https://mailchi.mp/helloswasthya.com/ojfpb0pcsp',
  'subject_line': 'स्तनपान पर कैसे असर डालती है डिलिवरी?',
  'title': '6th August Newsletter'},
 {'archive_url': 'http://eepurl.com/gCdCtD',
  'campaign_id': '811eaa1c87',
  'content_type': 'template',
  'country_code': 'MM',
  'create_date': '2019-09-03',
  'create_time': '2019-09-03T08:01:24+00:00',
  'list_id': '3d30d693ae',
  'list_name': 'Hello Sayarwon',
  'long_archive_url': 'https://us11.campaign-archive.com/?u=2f8c23d0988167d58c905ee8c&id=811eaa1c87',
  'subject_line': 'Posts from Hello Sayarwon for 09/03/2019',
  'title': 'Weekly Newsletter Testing'},
 {'archive_url': 'http://eepurl.com/g-A-t9',
  'campaign_id': '81348a3b56',
  'content_type': 'template',
  'country_code': 'ID',
  'create_date': 

"\nrun_hhg_mailchimp_all_content_di = PythonOperator(\n    task_id='hhg_mailchimp_all_content_di',\n    provide_context=False,\n    depends_on_past=False,\n    python_callable=streaming_to_bq,\n    op_kwargs={'table_id': 'hhg-ods.test.content', 'rows_to_insert': all_content, 'schema': content_schema}\n    dag=dag,\n)\n\u200b\n\u200b\n\u200b\nload_csv = gcs_to_bq.GoogleCloudStorageToBigQueryOperator(\n    task_id='gcs_to_bq',\n    bucket='hhg_mailchimp_campaign',\n    source_objects=['all_campaigns_di.csv'],\n    destination_project_dataset_table='hhg_lead_mailchimp.hhg_mailchimp_all_content_di',\n    job_config = bigquery.LoadJobConfig(),\n    job_config.allow_quoted_newlines = True,\n    job_config.schema_fields=[\n        {'name': 'campaign_id', 'type': 'STRING', 'mode': 'NULLABLE'},\n        {'name': 'create_date', 'type': 'DATE', 'mode': 'NULLABLE'},\n        {'name': 'plain_text', 'type': 'STRING', 'mode': 'NULLABLE'},\n        {'name': 'url', 'type': 'STRING', 'mode': 'NULLABLE'}